In [1]:
# In 01_data_exploration.ipynb
import pandas as pd
import numpy as np

# Load core tables
customers = pd.read_csv('../data/raw/olist_customers_dataset.csv')
orders = pd.read_csv('../data/raw/olist_orders_dataset.csv')
order_items = pd.read_csv('../data/raw/olist_order_items_dataset.csv')
products = pd.read_csv('../data/raw/olist_products_dataset.csv')

print("Customers:", customers.shape)
print("Orders:", orders.shape)
print("Order Items:", order_items.shape)
print("Products:", products.shape)

Customers: (99441, 5)
Orders: (99441, 8)
Order Items: (112650, 7)
Products: (32951, 9)


In [2]:
# Cell 1: Check data types and missing values
print("=== CUSTOMERS ===")
print(customers.info())
print("\nMissing values:", customers.isnull().sum())

print("\n=== ORDERS ===")
print(orders.info())
print("\nMissing values:", orders.isnull().sum())

print("\n=== ORDER ITEMS ===")
print(order_items.info())
print("\nMissing values:", order_items.isnull().sum())

=== CUSTOMERS ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB
None

Missing values: customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64

=== ORDERS ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id      

In [3]:
# Cell 2: Preview first few rows
print("Customers sample:")
display(customers.head(3))

print("\nOrders sample:")
display(orders.head(3))

print("\nOrder items sample:")
display(order_items.head(3))

Customers sample:


,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP



Orders sample:


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00



Order items sample:


,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.9,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.0,17.87


In [4]:
# Cell 3: Basic statistics
print("Order value statistics:")
print(order_items['price'].describe())

print("\nOrder timestamps range:")
print("Earliest:", orders['order_purchase_timestamp'].min())
print("Latest:", orders['order_purchase_timestamp'].max())

Order value statistics:
count    112650.000000
mean        120.653739
std         183.633928
min           0.850000
25%          39.900000
50%          74.990000
75%         134.900000
max        6735.000000
Name: price, dtype: float64

Order timestamps range:
Earliest: 2016-09-04 21:15:19
Latest: 2018-10-17 17:30:18


In [5]:
import sqlite3

# Create in-memory database
conn = sqlite3.connect(':memory:')

# Load data to SQL
customers.to_sql('customers', conn, index=False)
orders.to_sql('orders', conn, index=False)
order_items.to_sql('order_items', conn, index=False)

# Write your first SQL query
query = """
SELECT 
    c.customer_state,
    COUNT(DISTINCT o.order_id) as total_orders,
    SUM(oi.price) as total_revenue
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
JOIN order_items oi ON o.order_id = oi.order_id
GROUP BY c.customer_state
ORDER BY total_revenue DESC
LIMIT 5;
"""

result = pd.read_sql_query(query, conn)
print("Top 5 states by revenue:")
display(result)

Top 5 states by revenue:


,customer_state,total_orders,total_revenue
0,SP,41375,5202955.05
1,RJ,12762,1824092.67
2,MG,11544,1585308.03
3,RS,5432,750304.02
4,PR,4998,683083.76


In [11]:
query2 = """
SELECT 
    strftime('%Y-%m', o.order_purchase_timestamp) as month,
    COUNT(DISTINCT o.order_id) as order_count,
    SUM(oi.price) as total_revenue,
    AVG(oi.price) as avg_order_value
FROM orders o
JOIN order_items oi ON o.order_id = oi.order_id
WHERE o.order_purchase_timestamp IS NOT NULL
GROUP BY month
ORDER BY month
"""

result2 = pd.read_sql_query(query2, conn)
display(result2)

,month,order_count,total_revenue,avg_order_value
0,2016-09,3,267.36,44.560000
1,2016-10,308,49507.66,136.384738
2,2016-12,1,10.90,10.900000
3,2017-01,789,120312.87,125.982063
4,2017-02,1733,247303.02,126.757058
5,2017-03,2641,374344.30,124.781433
6,2017-04,2391,359927.23,134.101054
7,2017-05,3660,506071.14,122.357626
8,2017-06,3217,433038.60,120.859224
9,2017-07,3969,498031.48,110.208338


In [12]:
query3 = """
WITH customer_order_counts AS (
    SELECT 
        customer_id,
        COUNT(DISTINCT order_id) as order_count
    FROM orders
    GROUP BY customer_id
)
SELECT 
    CASE 
        WHEN order_count = 1 THEN 'One-time'
        WHEN order_count = 2 THEN 'Repeat (2)'
        WHEN order_count BETWEEN 3 AND 5 THEN 'Repeat (3-5)'
        ELSE 'Loyal (6+)'
    END as customer_segment,
    COUNT(*) as customer_count,
    ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER(), 2) as percentage
FROM customer_order_counts
GROUP BY customer_segment
ORDER BY customer_count DESC
"""

result3 = pd.read_sql_query(query3, conn)
display(result3)

,customer_segment,customer_count,percentage
0,One-time,99441,100.0


In [14]:
# Add products table to SQLite
products.to_sql('products', conn, index=False, if_exists='replace')

# Now run Query 4 again
result4 = pd.read_sql_query(query4, conn)
display(result4)

,product_category_name,order_count,total_revenue,avg_price
0,beleza_saude,8836,1258681.34,130.16
1,relogios_presentes,5624,1205005.68,201.14
2,cama_mesa_banho,9417,1036988.68,93.30
3,esporte_lazer,7720,988048.97,114.34
4,informatica_acessorios,6689,911954.32,116.51
5,moveis_decoracao,6449,729762.49,87.56
6,cool_stuff,3632,635290.85,167.36
7,utilidades_domesticas,5884,632248.66,90.79
8,automotivo,3897,592720.11,139.96
9,ferramentas_jardim,3518,485256.46,111.63


In [15]:
# Verify all tables exist
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", conn)
print("Available tables:", tables['name'].tolist())

Available tables: ['customers', 'orders', 'order_items', 'products']


In [16]:
query4 = """
SELECT 
    p.product_category_name,
    COUNT(DISTINCT oi.order_id) as order_count,
    SUM(oi.price) as total_revenue,
    ROUND(AVG(oi.price), 2) as avg_price
FROM order_items oi
JOIN products p ON oi.product_id = p.product_id
WHERE p.product_category_name IS NOT NULL
GROUP BY p.product_category_name
ORDER BY total_revenue DESC
LIMIT 10
"""


result4 = pd.read_sql_query(query4, conn)
display(result4)

,product_category_name,order_count,total_revenue,avg_price
0,beleza_saude,8836,1258681.34,130.16
1,relogios_presentes,5624,1205005.68,201.14
2,cama_mesa_banho,9417,1036988.68,93.30
3,esporte_lazer,7720,988048.97,114.34
4,informatica_acessorios,6689,911954.32,116.51
5,moveis_decoracao,6449,729762.49,87.56
6,cool_stuff,3632,635290.85,167.36
7,utilidades_domesticas,5884,632248.66,90.79
8,automotivo,3897,592720.11,139.96
9,ferramentas_jardim,3518,485256.46,111.63


In [18]:
print("Top categories (Portuguese):")
print(result4.head(10))

Top categories (Portuguese):
    product_category_name  order_count  total_revenue  avg_price
0            beleza_saude         8836     1258681.34     130.16
1      relogios_presentes         5624     1205005.68     201.14
2         cama_mesa_banho         9417     1036988.68      93.30
3           esporte_lazer         7720      988048.97     114.34
4  informatica_acessorios         6689      911954.32     116.51
5        moveis_decoracao         6449      729762.49      87.56
6              cool_stuff         3632      635290.85     167.36
7   utilidades_domesticas         5884      632248.66      90.79
8              automotivo         3897      592720.11     139.96
9      ferramentas_jardim         3518      485256.46     111.63


In [19]:
category_translation = {
    'beleza_saude': 'beauty_health',
    'informatica_acessorios': 'computers_accessories',
    'cama_mesa_banho': 'bed_bath_table',
    'esporte_lazer': 'sports_leisure',
    'moveis_decoracao': 'furniture_decor',
}

result4['category_english'] = result4['product_category_name'].map(
    lambda x: category_translation.get(x, x)
)
display(result4[['product_category_name', 'category_english', 'total_revenue']].head(10))

,product_category_name,category_english,total_revenue
0,beleza_saude,beauty_health,1258681.34
1,relogios_presentes,relogios_presentes,1205005.68
2,cama_mesa_banho,bed_bath_table,1036988.68
3,esporte_lazer,sports_leisure,988048.97
4,informatica_acessorios,computers_accessories,911954.32
5,moveis_decoracao,furniture_decor,729762.49
6,cool_stuff,cool_stuff,635290.85
7,utilidades_domesticas,utilidades_domesticas,632248.66
8,automotivo,automotivo,592720.11
9,ferramentas_jardim,ferramentas_jardim,485256.46
